# Convolution


## Exercise 03.2.1 1D Convolution

$$ O_{q} = I_{(q+s)} \times F_{s} $$

<img align="center" src="figures/03.2.1.conv1d_setup.png" alt="conv1d_setup" style="width:90%">


### Understanding the Inputs: Workload Specification


In [ ]:
%set_env BASELINE_DIR=../../../example_designs
%set_env DESIGNS_DIR=../../../example_designs/example_designs/sparseloop
%set_env SCRIPTS_DIR=../scripts

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.1-conv1d/ && cat problem.yaml

### Understanding Inputs: Architecture Specification

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.1-conv1d/ && cat arch.yaml

### Understanding the Inputs: Mapping Specification

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.1-conv1d/ && cat mapping.yaml

### Understanding the Inputs: Sparse Optimziation Specification
- Compressed representation format for Weights. Note we use run length encoding (RLE) this time.
- Skip Inputs and Outputs based on Weights

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.1-conv1d/ && cat arch.yaml

### Run Example

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/03.2.1-conv1d

### Examine Important Stats
- Only Weights data has associated metadata (and thus the savings and overhead brought by metadata).
- Inputs and Outputs are skipped according to Weight's sparsity.
- Reduced number of computes due to skipping at the `Buffer`.
- Overall similar behaviors compared to what we saw for the similar dot product example.

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/03.2.1-conv1d/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/03.2-conv-aggregated-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

## Conv1D with Output Channels


$$ O_{m,q} = I_{(q+s)} \times F_{m,s} $$

## Exercise 03.2.2 conv1D + Outputs Channels

### Understanding the Inputs: Workload Specification

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.2-conv1d+oc/ && cat problem.yaml

### Understanding the Inputs: Mapping Specification for New Workload


In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.2-conv1d+oc/ && cat mapping.yaml

### Understanding the Inputs: Sparse Optimization Specification
The extra output channel dimension introduces an extra rank in data representation.

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.2-conv1d+oc/ && cat arch.yaml

### Run Example

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/03.2.2-conv1d+oc

## Exercise 03.2.3 Conv1D + Output Channels- Spatial Tiling

### Understanding the Inputs: Architecture Specification

- Spatial fanout of the PEs for parallel processing
- Reduced the per-PE buffer size

<img align="center" src="figures/03.2.3.spatial_arch.png" alt="figures/03.2.3.spatial_arch.png" style="width:60%">

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.3-conv1d+oc-spatial/ && cat arch.yaml

### Understanding the Inputs: Mapping Specification

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.3-conv1d+oc-spatial/ && cat mapping.yaml

### Understanding the Inputs: Sparse Optimization Specification

Because of the spatial tiling, we need to pretile the tensor before sending it out for processing. We now have more ranks for Filter.

In [ ]:
%%bash
cd $DESIGNS_DIR/03.2.3-conv1d+oc-spatial/ && cat arch.yaml

### Run Example

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/03.2.3-conv1d+oc-spatial

### Examine Important Stats
- Pretiling overhead incurred by the temporal and spatial tiling.
- Reduced tile size stored in `Buffer`.
- Similar savings from explicit skipping as above.

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/03.2.3-conv1d+oc-spatial/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/03.2-conv-aggregated-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

### Compare Untiled Example and  Tiled Example

- Untiled exampled consumes more energy and has a slower processing speed
- Spatially tiled example consumes more area but has lower energy consumption and faster processing speed

In [ ]:
%%bash
export OUT_DIR_1=$(realpath ../../../example_designs/example_designs/sparseloop/03.2.2-conv1d+oc/outputs/default_problem)
export OUT_DIR_2=$(realpath ../../../example_designs/example_designs/sparseloop/03.2.3-conv1d+oc-spatial/outputs/default_problem)
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/03.2-conv-summary-stats.sh)

echo "========================================"
echo "             Untiled Example            "
echo "========================================"
chmod +x $SCRIPT_PATH && cd $OUT_DIR_1 && $SCRIPT_PATH

echo "========================================"
echo "         Tiled Example        "
echo "========================================"
chmod +x $SCRIPT_PATH && cd $OUT_DIR_2 && $SCRIPT_PATH
